# Benoit's Problem with various RTO Algorithms

In [29]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import approx_fprime

# Benoit Model (Modified Version) for Real Time Optimization without constraint

In [30]:
# Plant Model 
def Benoit_Model(u,theta):
    f = theta[0] * u[0] ** 2 + theta[1] * u[1] ** 2
    return f


def con1_model(u,theta):
    g1 = 1. - theta[2]*u[0] + theta[3]*u[1] ** 2
    return -g1

# Actual Plant System
def Benoit_System_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2 + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2. + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def Benoit_System_noiseless_1(u):
    f = u[0] ** 2 + u[1] ** 2 + u[0] * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return f

def Benoit_System_noiseless_2(u):
    f = u[0] ** 2 + u[1] ** 2 + (1 - u[0] * u[1])**2  # + np.random.normal(0., np.sqrt(1e-3))
    return f


def con1_system_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1] - 2.  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


def con1_system_tight_noiseless(u):
    g1 = 1. - u[0] + u[1] ** 2 + 2. * u[1]  # + np.random.normal(0., np.sqrt(1e-3))
    return -g1


# 1. Model Adaptation

## A. Optimization on cost function

Optimization algorithm on cost function to find optimized input u given parameter

In [31]:
# Optimization on cost function
def cost_optimize(theta,u0):
    con = ({'type': 'ineq', 
            'fun': lambda u: con1_model(u,theta)}) 
    result = minimize(Benoit_Model,
                    u0,
                    constraints= con,
                    method='SLSQP',
                    options={'ftol': 1e-9},
                    args= (theta))

    return result.x,result.fun

In [32]:
# Test
u0 = [1,1] # Initial guess for optimization algorithm
theta = [0.5,0.5,0.5,0.5]
u,fun = cost_optimize(theta,u0)
print(f"optimal input: {u}, optimal output: {fun}")


optimal input: [ 4.00000000e+00 -3.29554324e-08], optimal output: 7.999999999980859


## B. Model Adaptation

We try to minimize difference between output of an actual plant and output of a model. 

The difference is measured by SSE (Sum of Squared Error), which is used for cost function to be minimized

In [33]:
# cost function of adaptation
def SSE(theta,u,plant_func,plant_constraint):

    func_diff = plant_func(u) - Benoit_Model(u,theta)
    con_diff = plant_constraint(u) - con1_model(u,theta)

    return func_diff**2+con_diff**2

# Model Adaptation (Optimization of Parameters)
def SSE_optimize(plant_func,plant_constraint,theta0,u):

    result = minimize(SSE,
                      theta0,
                      method='SLSQP',
                      args= (u,plant_func,plant_constraint),
                      options= {'ftol': 1e-9})
    
    return result.x,result.fun


In [34]:
# test for adaptation cost function
x = SSE(theta = [0.5,0.5,0.5,0.5],
        u = [2,0],
        plant_func = Benoit_System_noiseless_1,
        plant_constraint =con1_system_noiseless)

print(f"cost function:{x}")
print("------------")

# test for model adaptation method
theta,fun = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                     plant_constraint=con1_system_noiseless,
                     theta0=[0.5,0.5,0.5,0.5],
                     u=[2,0])

print(f"optimal hyperparameters:{theta},\noptimal function: {fun}")



cost function:13.0
------------
optimal hyperparameters:[0.99999999 0.5        1.99999999 0.5       ],
optimal function: 1.1172258988382782e-15


## C. Overall Algorithm

### I. Model Adaptation on "Benoit_System_noiseless_1" plant system with "con1_system_noiseless" constraint

In [35]:
# Initial Guess (k=0)
uk = [1,1]
thetak = [0.5,0.5,0.5,0.5]

# dictionary: uk_1, fun, thetak_1, cost, 

for i in range(5):


    # Model Optimization
    uk_1,fun = cost_optimize(theta=thetak,u0=uk)
    print(f"optimal input: {uk_1}, optimal output: {fun}")
    
    # Model Adaptation
    thetak_1,cost = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                        plant_constraint=con1_system_noiseless,
                        theta0=thetak,
                        u=uk_1)
    print(f"optimal hyperparameters:{thetak_1}, cost: {cost}")

    uk = uk_1
    thetak = thetak_1




optimal input: [ 4.00000000e+00 -3.29554324e-08], optimal output: 7.999999999980859
optimal hyperparameters:[0.99999998 0.5        1.50000001 0.5       ], cost: 1.5727754259905706e-14
optimal input: [ 1.33333332e+00 -3.45879808e-07], optimal output: 1.7777777203419192
optimal hyperparameters:[0.99999954 0.5        2.50000053 0.5       ], cost: 1.3285434674341004e-13
optimal input: [ 7.99999830e-01 -1.92099932e-07], optimal output: 0.6399994309844345
optimal hyperparameters:[0.99999974 0.5        3.500001   0.5       ], cost: 2.280910296758929e-16
optimal input: [ 5.71428408e-01 -3.08785290e-08], optimal output: 0.3265303398567099
optimal hyperparameters:[0.99999983 0.5        4.50000109 0.5       ], cost: 1.5295708401470793e-15
optimal input: [ 4.44444337e-01 -8.49722276e-09], optimal output: 0.19753073527222617
optimal hyperparameters:[0.99999987 0.5        5.50000112 0.5       ], cost: 5.342817281437912e-16


### Actual Optimized Input and Output

In [36]:
u0 = [1,1] 
con = ({'type': 'ineq', 
        'fun': lambda u: con1_system_noiseless(u)}) 
result = minimize(Benoit_System_noiseless_1,
                u0,
                constraints= con,
                method='SLSQP',
                options={'ftol': 1e-9})

u = result.x
fun = result.fun

print(f"optimal input: {u}, optimal output: {fun}")

optimal input: [0. 0.], optimal output: 0.0


### Result Analysis

Notice that actual optimal input is [0,0] while the optimal input from model adaption converges to the model. However, the convergence is done with wrong hyperparameters especially on constraint con1_system_noiseless; notice constant increase in theta[2] (=47 at the last iteration). 

Moreover, the hyperparameters theta[1], theta[3] (=0.5,0.5) does not change because intermidiate optimal inputs have u[1] = 0 always. If the initial values were [0.1,0.1,0.1,0.1] then theta[1], theta[3] = 0.1 always. This makes hyperparameter to not get changed in model adaptation step.

### II. Model Adaptation on "Benoit_System_noiseless_1" plant system with "con1_system_tight_noiseless" constraint

In [37]:
# Initial Guess (k=0)
uk = [1,1]
thetak = [0.5,0.5,0.5,0.5]

# dictionary: uk_1, fun, thetak_1, cost, 

for i in range(5):


    # Model Optimization
    uk_1,fun = cost_optimize(theta=thetak,u0=uk)
    print(f"optimal input: {uk_1}, optimal output: {fun}")
    
    # Model Adaptation
    thetak_1,cost = SSE_optimize(plant_func=Benoit_System_noiseless_1,
                        plant_constraint=con1_system_tight_noiseless,
                        theta0=thetak,
                        u=uk_1)
    print(f"optimal hyperparameters:{thetak_1}, cost: {cost}")

    uk = uk_1
    thetak = thetak_1

optimal input: [ 4.00000000e+00 -3.29554324e-08], optimal output: 7.999999999980859
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 6.005331388506965e-15
optimal input: [ 1.99999997e+00 -5.99199530e-07], optimal output: 3.9999998408278277
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 2.6867529293183906e-12
optimal input: [ 1.99999997e+00 -5.99199530e-07], optimal output: 3.9999998408278277
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 2.6867529293183906e-12
optimal input: [ 1.99999997e+00 -5.99199530e-07], optimal output: 3.9999998408278277
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 2.6867529293183906e-12
optimal input: [ 1.99999997e+00 -5.99199530e-07], optimal output: 3.9999998408278277
optimal hyperparameters:[0.99999999 0.5        1.00000001 0.5       ], cost: 2.6867529293183906e-12


### Actual Optimized Input and Output

In [38]:
u0 = [1,1] 
con = ({'type': 'ineq', 
        'fun': lambda u: con1_system_tight_noiseless(u)}) 
result = minimize(Benoit_System_noiseless_1,
                u0,
                constraints= con,
                method='SLSQP',
                options={'ftol': 1e-9})

u = result.x
fun = result.fun

print(f"optimal input: {u}, optimal output: {fun}")

optimal input: [ 0.3684571  -0.39299332], optimal output: 0.14540320807022292


### Result Analysis

TO BE WRITTEN:

The result seems to show the lack of model flexibility, which prevents the optimal input from model to conver to actual optimal input. 

OR 

maybe due to that my code is not working

# 2. Modifier Adaptation

## A. Modifier

In [39]:
# bias modifier
def bias_modifier(u):
    epsil = Benoit_System_noiseless_1(u) - Benoit_Model(u,theta)
    return epsil

# gradient modifier
def gradient_modifier(u,du,func):
    
    # Predicted gradient from real plant
    gradient_p_u0 = (func(u+[du,0]) - func(u))/du
    gradient_p_u1 = (func(u+[0,du]) - func(u))/du
    gradient_p = [gradient_p_u0,gradient_p_u1]

    # Gradient from model plant
    gradient_m = approx_fprime(u,Benoit_Model,epsilon=1e-6,args=(theta))
    
    # Difference between gradient of a plant and gradient of a plant model
    lamda = [gradient_p - gradient_m]
    return lamda

## B. Optimization on Cost Function

In [40]:
# YET TO BE MODIFIED

# Optimization on cost function
def cost_optimize(theta,u0):
    con = ({'type': 'ineq', 
            'fun': lambda u: con1_model(u,theta)}) 
    result = minimize(Benoit_Model,
                    u0,
                    constraints= con,
                    method='SLSQP',
                    options={'ftol': 1e-9},
                    args= (theta))

    return result.x,result.fun